# generatingFakeReadsForTesting.ipynb
## Marcus Viscardi,    September 02, 2022

I want to try and do some simple read generation with just text "parsing" and the expected read sequences

The main goal would be to make a simple function that could take an expected read sequence identity and then spit out a crapload of altered reads that could be used for testing and such!

In [1]:
import mappy as mp
from pprint import pprint
import pandas as pd
from tqdm import tqdm
from typing import Union, List
from nanoporePipelineCommon import find_newest_matching_file, pick_libs_return_paths_dict, get_dt

pd.set_option("display.max_columns", None)
print("imports done")

imports done


In [8]:
finalStandards_dict = {}
for read_id, sequence, quality, comments in mp.fastx_read("/data16/marcus/scripts/nanoporePipelineScripts/standardsAlignment/220902_version2.0_releventSequences.fasta", read_comment=True):
    if "finalStandard" in read_id:
        finalStandards_dict[read_id] = (sequence, comments)

In [12]:
def sequence_subindel_creater(seq,
                              insert_prob=0.01,
                              delete_prob=0.01,
                              sub_prob=0.05) -> str:
    # TODO: Add ability to output multiple sequences
    # TODO: Add adjustable propensity to truncate reads!
    seq = seq.upper()
    adjusted_seq = ""
    choices_per_nt = {"insert": insert_prob,
                      "delete": delete_prob,
                      "substitute": sub_prob}
    for 

ENO2_finalStandard_00Tail
     662bp 662
ENO2_finalStandard_05Tail
     667bp 667
ENO2_finalStandard_10Tail
     672bp 672
ENO2_finalStandard_15Tail
     677bp 677
ENO2_finalStandard_30Tail
     692bp 692
ENO2_finalStandard_60Tail
     722bp 722
